Customer MDM Data Standardization 

Here, focus is on Address standardization, Email Standardization and Phone number standardization using various open source libraries 

1) Address Validation 
Various open source modules in python were explored - pyzipcode,uszipcode etc but none were complete or showed promising results. Split into State,City and Zipcode validation 

Zipcode Database - https://www.unitedstateszipcodes.org/zip-code-database/

Since this Database had state in abbreviated form whereas input file contain full state names, created a separate lookup file with (State Name, Abbreviated) and combined the 2 files to be used for lookups.

Zip code -- validate 5 digit else put both city and state as NA 


2) Email Validation 
Pre processing: --> Create a lookup file with all valid values for Top Level Domains(tld) #file based on 'http://www.iana.org/domains/root/db', can build a web scraper in a job to update the file 
Step 1: validate that email has only one '@' sign and atleast one '.' after the '@' sign 
Step 2: Extract tld and ensure it is valid 

3) Phonenumber Validation 
Using Libphonenumbers python package 
1) Validate that the phone is valid eg: for US 10 digits without the '+1' or 11 digits total 
2) Standardize into E164 format (EG: +13346754333) 



In [126]:
import numpy as np 
import pandas as pd


In [127]:
file_name = "data_bank_accounts.csv"
bank_cust_data = pd.read_csv(file_name)

In [128]:
bank_cust_data.head(15)  #this is the input file 

,source_rowID,first_name,last_name,street,city,state,zip,phone,email
0,1,Dennis,Sullivan,232 Dryden Terrace,Appleton,12345,654321,334-6754-333,dsullivan1@ucla.edu
1,2,Harry,Davis,123 ABC,New York City,New York,55555,1-(999)999-99999,hdavis3@networkadvertising.org
2,3,Michelle,White,185 Manitowish Point,Atlanta,Georgia,30323,1-(678)692-6250,mwhite4@hud.gov
3,4,Marie,Wood,98869 Claremont Place,Pasadena,California,91125,1-(626)747-3764,mwood5@bravesites.com
4,5,Karen,Fisher,4323 Bluestem Center,Philadelphia,Pennsylvania,19115,1-(267)279-4286,kfishere@blogger
5,6,Gary,Carroll,678 Wayridge Center,Scranton,Pennsylvania,18514,1-(570)785-1034,g.carrollg@discovery.com
6,7,Paul,Gonzales,47 Main Crossing,Pittsburgh,Pennsylvania,15205,1-(724)209-2821,pgonzalesh@pbs.pqrs
7,8,Rose,Sims,03535 Knutson Place,Hattiesburg,Mississippi,39404,1-(601)773-3815,rsimsi@slideshare.net
8,316,Ronald,Powell,2 Crowley Center,Pasadena,California,91131,1-(626)574-7779,rpowell0@jomla.org
9,317,Judy,Harrison,59517 Dottie Way,Birmingham,Alabama,35263,1-(205)308-3808,jharrison7@sogou.com


In [157]:
#this is the final output
print(bank_cust_data)

    source_rowID first_name  last_name                 street           city  \
0              1     Dennis   Sullivan     232 Dryden Terrace       Appleton   
1              2      Harry      Davis                123 ABC  New York City   
2              3   Michelle      White   185 Manitowish Point        Atlanta   
3              4      Marie       Wood  98869 Claremont Place       Pasadena   
4              5      Karen     Fisher   4323 Bluestem Center   Philadelphia   
5              6       Gary    Carroll    678 Wayridge Center       Scranton   
6              7       Paul   Gonzales       47 Main Crossing     Pittsburgh   
7              8       Rose       Sims    03535 Knutson Place    Hattiesburg   
8            316     Ronald     Powell       2 Crowley Center       Pasadena   
9            317       Judy   Harrison       59517 Dottie Way     Birmingham   
10           318     Andrew  Robertson       9812 Troy Avenue        Houston   
11           319       Ruth   Gonzales  

In [130]:
#below is the detailed steps and code for converting input into the output
#address validation

zip_file = 'zip_code_database.csv' #https://www.unitedstateszipcodes.org/zip-code-database/
#zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2014
city_state_zip = pd.read_csv(zip_file,encoding='latin-1')


In [131]:
city_state_zip.head(10)

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2014
0,501,UNIQUE,0,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Irs Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,NaN,"787,939",NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0
5,604,PO BOX,0,Aguadilla,Ramey,NaN,PR,NaN,NaN,NaN,NaN,US,18.43,-67.15,0
6,605,PO BOX,0,Aguadilla,NaN,NaN,PR,NaN,NaN,NaN,NaN,US,18.43,-67.15,0
7,606,STANDARD,0,Maricao,NaN,Urb San Juan Bautista,PR,Maricao Municipio,America/Puerto_Rico,"787,939",NaN,US,18.18,-66.98,0
8,610,STANDARD,0,Anasco,NaN,"Brisas De Anasco, Est De Valle Verde, Jard De ...",PR,Añasco Municipio,NaN,787,NaN,US,18.28,-67.14,0
9,611,PO BOX,0,Angeles,NaN,NaN,PR,NaN,NaN,NaN,NaN,US,18.28,-66.79,0


In [132]:
addr_filtered = city_state_zip.filter(items=['zip','primary_city','state'])
print(addr_filtered)

         zip     primary_city state
0        501       Holtsville    NY
1        544       Holtsville    NY
2        601         Adjuntas    PR
3        602           Aguada    PR
4        603        Aguadilla    PR
5        604        Aguadilla    PR
6        605        Aguadilla    PR
7        606          Maricao    PR
8        610           Anasco    PR
9        611          Angeles    PR
10       612          Arecibo    PR
11       613          Arecibo    PR
12       614          Arecibo    PR
13       616         Bajadero    PR
14       617      Barceloneta    PR
15       622         Boqueron    PR
16       623        Cabo Rojo    PR
17       624         Penuelas    PR
18       627            Camuy    PR
19       631         Castaner    PR
20       636          Rosario    PR
21       637    Sabana Grande    PR
22       638           Ciales    PR
23       641           Utuado    PR
24       646           Dorado    PR
25       647         Ensenada    PR
26       650          Florid

In [133]:
addr_filtered.dtypes

zip              int64
primary_city    object
state           object
dtype: object

In [134]:
#filter out where zip code not 5 digit 

filtered = addr_filtered[addr_filtered['zip'].map(str).map(len) == 5]

In [135]:
filtered.head(10)

,zip,primary_city,state
3690,10001,New York,NY
3691,10002,New York,NY
3692,10003,New York,NY
3693,10004,New York,NY
3694,10005,New York,NY
3695,10006,New York,NY
3696,10007,New York,NY
3697,10008,New York,NY
3698,10009,New York,NY
3699,10010,New York,NY


In [136]:
#merge with a new file to get full State Name
state_abbr_file ='state_abbr.csv'
state_abbr = pd.read_csv(state_abbr_file)

In [137]:
state_abbr.head(55)

,State,Abbreviation
0,ALABAMA,AL
1,ALASKA,AK
2,ARIZONA,AZ
3,ARKANSAS,AR
4,CALIFORNIA,CA
5,COLORADO,CO
6,CONNECTICUT,CT
7,DELAWARE,DE
8,FLORIDA,FL
9,GEORGIA,GA


In [138]:
addr_with_state=pd.merge(filtered, state_abbr, left_on=  ['state'],right_on= ['Abbreviation'], how = 'left')

In [139]:
addr_with_state.head(10)

,zip,primary_city,state,State,Abbreviation
0,10001,New York,NY,NEW YORK,NY
1,10002,New York,NY,NEW YORK,NY
2,10003,New York,NY,NEW YORK,NY
3,10004,New York,NY,NEW YORK,NY
4,10005,New York,NY,NEW YORK,NY
5,10006,New York,NY,NEW YORK,NY
6,10007,New York,NY,NEW YORK,NY
7,10008,New York,NY,NEW YORK,NY
8,10009,New York,NY,NEW YORK,NY
9,10010,New York,NY,NEW YORK,NY


In [140]:
series = bank_cust_data['zip']
from pandas import Series
data_zipcodes = []
data_cities = []
data_states = []

index1 = series.index


#Assuming our zipcode database is accurate, and using the "zip" field from source as a starting point we can validate the 
# combination of zip,city and state

for index, zipcode in series.items():
        if len(str(zipcode)) == 5:
                zipobj = addr_with_state[addr_with_state.zip == zipcode]
                if(zipobj.empty == True):
                    correct_city = 'Not_in_ZipCodeDB'
                    correct_state = 'Not_in_ZipCodeDB'
                else:
                    for index, row in zipobj.iterrows():
                            correct_city = row['primary_city']
                            correct_state = row['State']
                            break
        else: 
            correct_city = 'N/A'
            correct_state = 'N/A'
        data_cities.append(correct_city)
        data_states.append(correct_state)
        
cities_series = Series(data_cities,index =index1)
states_series = Series(data_states,index =index1)

print(cities_series)
print(states_series)

0                  N/A
1        Young America
2     Not_in_ZipCodeDB
3             Pasadena
4         Philadelphia
5             Scranton
6           Pittsburgh
7          Hattiesburg
8             Pasadena
9           Birmingham
10             Houston
11            New York
12         Port Orange
13              London
14            Scranton
dtype: object
0                  N/A
1            MINNESOTA
2     Not_in_ZipCodeDB
3           CALIFORNIA
4         PENNSYLVANIA
5         PENNSYLVANIA
6         PENNSYLVANIA
7          MISSISSIPPI
8           CALIFORNIA
9              ALABAMA
10               TEXAS
11            NEW YORK
12             FLORIDA
13            KENTUCKY
14        PENNSYLVANIA
dtype: object


In [141]:
bank_cust_data["correct_city"] = cities_series
bank_cust_data["correct_state"] = states_series

In [142]:
print(bank_cust_data) #Here we see the correct city and state abbreviations, 'N/A' occurs if zip code is not 5 digit and Not_in_ZipCodeDB means that the lookup database does not contain a value 

    source_rowID first_name  last_name                 street           city  \
0              1     Dennis   Sullivan     232 Dryden Terrace       Appleton   
1              2      Harry      Davis                123 ABC  New York City   
2              3   Michelle      White   185 Manitowish Point        Atlanta   
3              4      Marie       Wood  98869 Claremont Place       Pasadena   
4              5      Karen     Fisher   4323 Bluestem Center   Philadelphia   
5              6       Gary    Carroll    678 Wayridge Center       Scranton   
6              7       Paul   Gonzales       47 Main Crossing     Pittsburgh   
7              8       Rose       Sims    03535 Knutson Place    Hattiesburg   
8            316     Ronald     Powell       2 Crowley Center       Pasadena   
9            317       Judy   Harrison       59517 Dottie Way     Birmingham   
10           318     Andrew  Robertson       9812 Troy Avenue        Houston   
11           319       Ruth   Gonzales  

In [143]:
valid_csz_series =[]
for index, row in bank_cust_data.iterrows() :
    if((row['city'].upper()==row['correct_city'].upper()) & (row['state'].upper() == row['correct_state'].upper())):
        valid_csz_series.append(1)
    else:
        valid_csz_series.append(0)


        


In [144]:
print(valid_csz_series)

[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1]


In [145]:
address_flag_series = Series(valid_csz_series,index =index1)
bank_cust_data['valid_csz_flag']=address_flag_series

In [146]:
print(bank_cust_data)

    source_rowID first_name  last_name                 street           city  \
0              1     Dennis   Sullivan     232 Dryden Terrace       Appleton   
1              2      Harry      Davis                123 ABC  New York City   
2              3   Michelle      White   185 Manitowish Point        Atlanta   
3              4      Marie       Wood  98869 Claremont Place       Pasadena   
4              5      Karen     Fisher   4323 Bluestem Center   Philadelphia   
5              6       Gary    Carroll    678 Wayridge Center       Scranton   
6              7       Paul   Gonzales       47 Main Crossing     Pittsburgh   
7              8       Rose       Sims    03535 Knutson Place    Hattiesburg   
8            316     Ronald     Powell       2 Crowley Center       Pasadena   
9            317       Judy   Harrison       59517 Dottie Way     Birmingham   
10           318     Andrew  Robertson       9812 Troy Avenue        Houston   
11           319       Ruth   Gonzales  

In [147]:
#email validation 
tld_file = 'tld.rtf' #file based on 'http://www.iana.org/domains/root/db', can build a web scraper in a job to update the file 
email_series = bank_cust_data["email"]

tld_list = []
open_tld_file = open(tld_file,'r')

for x in open_tld_file:
    if (str(x).strip()!=''):
        tld_list.append(str(x).strip().strip('\\'))

len_list = []
for y in tld_list:
    len_list.append(len(y))




In [148]:
#funtion to find index of character in a string
def find_characters(word, character):
    found = []
    last_index = -1
    while True:
        try:
            last_index = word.index(character, last_index+1)
        except ValueError:
            break
        else:
            found.append(last_index)
    return found

In [149]:
valid_email_format =[]
valid_top_level_domain=[]
for ind,email in email_series.items():
    #parse email 
    #check if only 1 '@' 
    email_str = str(email)
    count_at = email_str.count('@')
    if count_at == 1:
        at_flag = True
          #find index of '@'
        index_at = email_str.index('@')
    else:
        at_flag = False
    #find count and indices of '.'s
    count_dot = email_str.count('.')
    dot_indices = find_characters(email_str,'.') #this is a list 
    for dot_ind in dot_indices:
        if(dot_ind > index_at):
            dot_flag = True
            break
        else:
            dot_flag= False
            
    if(at_flag==True and dot_flag==True and count_dot>0):
        valid_email_format.append(True)
    else:
        valid_email_format.append(False)
        
    #Extract Top Level Domain 
    if(count_dot>0):
        tld_dot_start_index = max(dot_indices)
        tld = email[(tld_dot_start_index +1):]
        if(tld.upper() in tld_list):
            valid_top_level_domain.append(True)
        else:
            valid_top_level_domain.append(False)
    else:
        valid_top_level_domain.append(False)

In [150]:
print(valid_email_format)

[True, True, True, True, False, True, True, True, True, True, True, True, True, True, True]


In [151]:
bank_cust_data["valid_email_format"]=valid_email_format

In [152]:
print(valid_top_level_domain)

[True, True, True, True, False, True, False, True, True, True, True, True, True, True, True]


In [153]:
bank_cust_data["valid_tld"]=valid_top_level_domain

In [154]:
print(bank_cust_data) #here we can see flags for valid email format and valid top level domain 

    source_rowID first_name  last_name                 street           city  \
0              1     Dennis   Sullivan     232 Dryden Terrace       Appleton   
1              2      Harry      Davis                123 ABC  New York City   
2              3   Michelle      White   185 Manitowish Point        Atlanta   
3              4      Marie       Wood  98869 Claremont Place       Pasadena   
4              5      Karen     Fisher   4323 Bluestem Center   Philadelphia   
5              6       Gary    Carroll    678 Wayridge Center       Scranton   
6              7       Paul   Gonzales       47 Main Crossing     Pittsburgh   
7              8       Rose       Sims    03535 Knutson Place    Hattiesburg   
8            316     Ronald     Powell       2 Crowley Center       Pasadena   
9            317       Judy   Harrison       59517 Dottie Way     Birmingham   
10           318     Andrew  Robertson       9812 Troy Avenue        Houston   
11           319       Ruth   Gonzales  

In [155]:
#phone validation 
import re 
import phonenumbers
phone_series = bank_cust_data["phone"]
formatted_phone_series=[]
valid_phone_series = []
for ind, phone_no in phone_series.items():
    phone_str = str(phone_no)
    phone_stripped = re.sub(r'[^0-9 ]',r'',phone_str)
    #print(phone_no,phone_stripped)  
    if(len(phone_stripped) ==10):
        phone_stripped = '1'+phone_stripped
    phone_stripped = '+'+phone_stripped
    #print(phone_no,phone_stripped) 
    p_parsed=phonenumbers.parse(phone_stripped,None)
    formatted_phone_series.append(phonenumbers.format_number(p_parsed, phonenumbers.PhoneNumberFormat.E164))
    if(phonenumbers.is_possible_number(p_parsed) and phonenumbers.is_valid_number(p_parsed)):
        valid_phone_series.append(True)
    else:
        valid_phone_series.append(False)
bank_cust_data["formatted_phone"] = formatted_phone_series
bank_cust_data["valid_phone_flag"] = valid_phone_series



In [156]:
print(bank_cust_data) #formatted_phone and valid_phone 

    source_rowID first_name  last_name                 street           city  \
0              1     Dennis   Sullivan     232 Dryden Terrace       Appleton   
1              2      Harry      Davis                123 ABC  New York City   
2              3   Michelle      White   185 Manitowish Point        Atlanta   
3              4      Marie       Wood  98869 Claremont Place       Pasadena   
4              5      Karen     Fisher   4323 Bluestem Center   Philadelphia   
5              6       Gary    Carroll    678 Wayridge Center       Scranton   
6              7       Paul   Gonzales       47 Main Crossing     Pittsburgh   
7              8       Rose       Sims    03535 Knutson Place    Hattiesburg   
8            316     Ronald     Powell       2 Crowley Center       Pasadena   
9            317       Judy   Harrison       59517 Dottie Way     Birmingham   
10           318     Andrew  Robertson       9812 Troy Avenue        Houston   
11           319       Ruth   Gonzales  